Com not available between 30/04/2019 21/10/2019. they inroduced robot check.

In [23]:
from bs4 import BeautifulSoup as beauty
import pandas as pd
import numpy as np
import os
import re
import datetime as dt

In [2]:
list_2019_Vker_Hu =[i for i in os.listdir() if '2019' in i if 'ingatlanokHU_Vker' in i]
list_2019_Vker_Com = [i for i in os.listdir() if '2019' in i if 'ingatlanCOM_Vker' in i]
list_2019_Vker_Cen = [i for i in os.listdir() if '2019' in i if 'otthoncentrum_Vker' in i]

In [3]:
HU = pd.DataFrame(columns =['id','sqm','room','type','price','description','agency','url','date'])

In [4]:
for i in list_2019_Vker_Hu:
    html = open(i, 'r').read()
    l = beauty(html, 'lxml')
    for ii in range(len(l.find_all('div', attrs={'class': 'result-row'}))):
        value = []
        # ID
        value.append(l.find_all('div', attrs={'class': 'result-row'})[ii].find_all('a')[1]['href'].split('/')[-1])
        #sqm
        sqm = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class':'column-first'})
        if len(sqm) > 0:
            value.append(sqm[0].text.strip().replace('\n', ' ').split(' ')[1])
        else: value.append(np.NAN)
        #rooms
        room = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class':'column'})
        if len(room) > 0:
            value.append(room[0].text.strip().replace('\n', ' ').split(' ')[1])
        else: value.append(np.NAN)
        # type
        types = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class':'column'})
        if len(types) > 1:
            value.append(types[1].text.strip().replace('\n', ' ').split(' ')[2])
        else: value.append(np.NAN)
        # price
        p = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class':'result-price left'})
        if len(p) > 0:
            value.append(p[0].text.strip().replace('.', '').split(' ')[0])
        else: value.append(np.NAN)
        #description
        d = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class': 'result-short-description'})
        if len(d) > 0:
            value.append(d[0].text.strip())
        else: value.append(np.NAN)
        #agency
        age = l.find_all('div', attrs={'class': 'result-row'})[ii].find_all(attrs={'class':'office-logo-container right'})
        if len(age) > 0:
            value.append(age[0].find('a')['title'])
        else: value.append(np.NAN)
        #url
        value.append(l.find_all('div', attrs={'class': 'result-row'})[0].find_all('a')[1]['href'])         
        #date     
        value.append(dt.datetime.date(dt.datetime.strptime(re.sub(r'\D', '', i), '%d%m%Y')))
        
        HU.loc[len(HU), :] =  value

In [5]:
HU['sqm'] = HU['sqm'].astype(float)
HU['room'] = HU['room'].astype(float)
HU['price'] = HU['price'].astype(float)

In [8]:
HU.to_csv('HU_Vker1809-2110.csv', index=False, index_label= None)

In [6]:
HU.groupby('date').agg({'id': 'count', 
                       'sqm': 'mean', 
                       'price': 'mean', 
                       'price': ['median','min','max']})

id        sqm       price                        
           count       mean      median         min         max
date                                                           
2019-09-18    26  30.346154  39900000.0  22700000.0  45003600.0
2019-09-24    36  30.343750  37500000.0  22700000.0  44300000.0
2019-09-30    32  30.468750  39900000.0  22700000.0  44300000.0
2019-10-10    33  30.151515  37500000.0  22700000.0  44300000.0
2019-10-14    36  30.361111  38700000.0  22700000.0  44300000.0
2019-10-21    26  30.576923  39900000.0  22700000.0  44300000.0

In [7]:
Cen = pd.DataFrame(columns = ['id','sqm','room','type','price','description','agency','url','region','date'])

In [52]:
for i in list_2019_Vker_Cen:
    html = open(i,'r').read()
    l = beauty(html, 'lxml')
    for ii in l.find_all('div', attrs={'class': 'estate-list-box'}):
        value = []
        # ID
        value.append(ii.find_all(attrs ={'class':'oc-icon icon-oc-oc-kedvenc estate-bookmark pls-login atRealEstateList-estateBookmark estate-add-localStorage'})[0]['data-estate-id'])
        #sqm
        sqm = ii.find_all(attrs={'class':'estate-size col-xs-6 text-right'})
        if len(sqm) > 0:
            value.append(sqm[0].text.strip().split(' ')[0])
        else: value.append(np.NAN)
        #rooms
        room = ii.find_all(attrs={'class':'atEstateListItem-Rooms'})
        if len(room) > 0:
            value.append(room[0].text.strip())
        else: value.append(np.NAN)
        # type
        types = ii.find_all(attrs={'class':'estate-type-size'})
        if len(types) > 0:
            value.append(types[0].text.strip().split(' ')[0])
        else: value.append(np.NAN)
        # price
        p = ii.find_all(attrs={'class':'estate-price atEstateListItem-Price col-xs-6'})
        if len(p) > 0:
            value.append(p[0].text.strip().replace(',', '.').split(' ')[0])
        else: value.append(np.NAN)
        #description
        d = ii.find_all(attrs={'class': 'inner-hide-description'})
        if len(d) > 0:
            value.append(d[0].text.strip())
        else: value.append(np.NAN)
        #agency
        age = ii.find_all(attrs={'class':'office-logo-container right'})
        if len(age) > 0:
            value.append(age[0].find('a')['title'])
        else: value.append(np.NAN)
        #url
        value.append('https://www.oc.hu'+ii.find_all('a')[0]['href'])  
        #region
        value.append(ii.find_all(attrs ={'class':'oc-icon icon-oc-oc-kedvenc estate-bookmark pls-login atRealEstateList-estateBookmark estate-add-localStorage'})[0]['data-estate-regno'])
        #date     
        value.append(dt.datetime.date(dt.datetime.strptime(re.sub(r'\D', '', i), '%d%m%Y')))
        
        Cen.loc[len(Cen), :] =  value

In [49]:
'https://www.oc.hu'+ii.find_all('a')[0]['href']

'https://www.oc.hu/ingatlanok/H393835?c=1b06e8e0'

In [54]:
Cen.to_csv('Cen_Vker1809-2110.csv', index=False, index_label=None)

In [55]:
pd.concat([HU,Cen], axis = 0)

/home/trismegistos/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,agency,date,description,id,price,region,room,sqm,type,url
0,CDC Ingatlan IV.Ker. Szita Ingatlan Kft.,2019-09-18,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",9721878,3.99e+07,NaN,1,32,Tégla,https://ingatlanok.hu/elado/lakas/budapest_v_k...
1,CDC Ingatlan IV.Ker. Szita Ingatlan Kft.,2019-09-18,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",9729036,3.75e+07,NaN,1,30,Tégla,https://ingatlanok.hu/elado/lakas/budapest_v_k...
2,Cdci Teréz krt-i iroda,2019-09-18,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",9721669,3.99e+07,NaN,1,32,Tégla,https://ingatlanok.hu/elado/lakas/budapest_v_k...
3,GDN Ingatlaniroda.,2019-09-18,EGYETEMISTÁK - BEFEKTETŐK- ELSŐ LAKÁSUKAT KERE...,9630578,4.43e+07,NaN,1,33,Tégla,https://ingatlanok.hu/elado/lakas/budapest_v_k...
4,CDC Báthory,2019-09-18,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",9721732,3.99e+07,NaN,1,32,Tégla,https://ingatlanok.hu/elado/lakas/budapest_v_k...
5,Tecnocasa,2019-09-18,"A belváros szívében, teljesen felújított, álló...",9739324,2.29e+07,NaN,1,25,Nincs,https://ingatlanok.hu/elado/lakas/budapest_v_k...
6,Tecnocasa,2019-09-18,"Váci utca sarkán, frekventált helyen, jó állap...",9738993,3.35e+07,NaN,1,29,Nincs,https://ingatlanok.hu/elado/lakas/budapest_v_k...
7,NaN,2019-09-18,"Budapest, 5th district, Vaci street, 30 m2 135...",9739093,4.50036e+07,NaN,1,30,Nincs,https://ingatlanok.hu/elado/lakas/budapest_v_k...
8,NaN,2019-09-18,"Budapest, 5th district, Vaci street, 30 m2 130...",9738947,4.33368e+07,NaN,1,30,Nincs,https://ingatlanok.hu/elado/lakas/budapest_v_k...
9,CDC Ingatlan IV.Ker. Szita Ingatlan Kft.,2019-09-18,"3 az 1-ben! Tökéletes befektetés, ahol a moder...",9729336,3.99e+07,NaN,1,32,Tégla,https://ingatlanok.hu/elado/lakas/budapest_v_k...
